In [14]:
with open('aoc17_input.txt') as f:
    lines = f.readline()
puzzle_input = lines.strip()

with open('aoc17_example.txt') as f:
    lines = f.readline()
example = lines.strip()

In [15]:
len(puzzle_input)

10091

In [25]:
import itertools
import numpy as np
from tqdm import tqdm
def has_overlap(list1,list2):
    overlap = [value for value in list1 if value in list2]
    return True if overlap else False

shape_points = {'square':[(0,0),(0,1),(1,0),(1,1)],
               'vert': [(0,0),(0,1),(0,2),(0,3)],
               'backL':[(0,0),(1,0),(2,0),(2,1),(2,2)],
               'plus':[(0,1),(1,0),(1,1),(2,1),(1,2)],
               'horiz':[(0,0),(1,0),(2,0),(3,0)]}
directions = {'<':-1,'>':1}
def dir_num(x):
    return int(directions[x])

In [26]:
example_list = [i for i in example]
example_list = list(map(dir_num,example_list))
example_list

puzz_list = [i for i in puzzle_input]
puzz_list = list(map(dir_num,puzz_list))
puzz_list

[1,
 1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 1,
 1,
 1,
 -1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 1,
 1,
 1,
 -1,
 -1,
 -1,
 1,
 1,
 1,
 -1,
 -1,
 1,
 1,
 1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 -1,
 1,
 1,
 1,
 -1,
 1,
 -1,
 1,
 1,
 -1,
 -1,
 -1,
 1,
 1,
 -1,
 1,
 -1,
 -1,
 1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 1,
 1,
 1,
 1,
 -1,
 -1,
 1,
 1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 1,
 -1,
 1,
 1,
 -1,
 1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 1,
 -1,
 1,
 1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 1,
 1,
 1,
 -1,
 -1,
 1,
 -1,
 1,
 1,
 1,
 1,
 -1,
 -1,
 -1,
 1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 1,
 1,
 1,
 -1,
 -1,
 -1,
 1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,

In [34]:
class Tetri:
    def __init__(self,x,y,shape):
        self.x = x
        self.y = y
        self.shape = shape
        self.blocks_relative = shape_points[self.shape]
        self.resting = False
        self.coords = [(self.x+b[0], self.y+b[1]) for b in self.blocks_relative]
        self.left = min([coord[0] for coord in self.coords])
        self.right = max([coord[0] for coord in self.coords])
        self.top = max([coord[1] for coord in self.coords])
        self.bottom = min([coord[1] for coord in self.coords])
        
    def update_coords(self):
        self.coords = [(self.x+b[0], self.y+b[1]) for b in self.blocks_relative]
        self.left = min([coord[0] for coord in self.coords])
        self.right = max([coord[0] for coord in self.coords])
        self.top = max([coord[1] for coord in self.coords])
        self.bottom = min([coord[1] for coord in self.coords])
    
    def project_coords(self,x1,y1):
        return [(x1 + b[0], y1+b[1])for b in self.coords]

In [35]:
class Grid:
    def __init__(self,width):
        self.width = width +1
        self.floor = 0
        self.height = 4
        
    def walls(self):
        return [(0,h) for h in range(1,self.height+1)] + [(self.width,h) for h in range(1,self.height+1)]
    
    def floors(self):
        return [(w,0) for w in range(1,self.width)]
    
    def corners(self):
        return [(0,0),(self.width,0)]
    
    def entry_square(self):
        return (3,self.height)

In [36]:
class TetrisGame:
    def __init__(self,gridsize,jets):
        self.grid = Grid(gridsize)
        self.resting_pieces = []
        self.pieces_dropped = 0
        self.current_piece = None
        self.highest_piece = None
        self.shape_order = itertools.cycle(['horiz','plus','backL','vert','square'])
        self.jets = itertools.cycle(jets)
        self.num_jets = 0
        self.len_jets = len(jets)
        self.cycle_states = []
        
    def cycle_state(self):
        return (self.current_piece.shape,self.current_piece.x,self.num_jets % self.len_jets)
        
    def do_turn(self):
        jet = next(self.jets)
        self.num_jets += 1
        if not (self.overlapping_walls(jet) or self.overlapping_resting_pieces((jet,0))):
            self.current_piece.x += jet
            self.current_piece.update_coords()
        if not self.overlapping_resting_pieces((0,-1)):
            self.current_piece.y -= 1
            self.current_piece.update_coords()
        else:
            self.current_piece.resting = True
            self.cycle_states.append(self.cycle_state())
        
    def overlapping_walls(self,jet):
        return (self.current_piece.left + jet <= 0) or (self.current_piece.right + jet >= self.grid.width)
    
    def overlapping_resting_pieces(self,dir_coord):
        if  self.resting_pieces:
            resting_piece_squares = []
            for piece in self.resting_pieces:
                resting_piece_squares.extend(piece.coords)
            return has_overlap(self.current_piece.project_coords(dir_coord[0],dir_coord[1]),resting_piece_squares)
        else:
            return (self.current_piece.bottom -1 <= 0)
    def add_piece(self):
        self.current_piece = Tetri(3,self.grid.height,next(self.shape_order))
        if self.pieces_dropped == 0:
            self.highest_piece = self.current_piece
        
    def drop_piece(self):
        self.add_piece()
        while not self.current_piece.resting:
            self.do_turn()
        self.resting_pieces.append(self.current_piece)
        if self.current_piece.top > self.highest_piece.top:
            self.highest_piece = self.current_piece
        self.current_piece = None
        self.pieces_dropped += 1
        self.maybe_extend_grid()
#         self.remove_stale_bricks()

    def maybe_extend_grid(self):
        if self.grid.height < self.highest_piece.top + 4:
            self.grid.height = self.highest_piece.top + 4
            
    def remove_stale_bricks(self):
        for piece in self.resting_pieces:
            if piece.y < self.grid.height - 100:
                self.resting_pieces.remove(piece)
                
    def draw_gameboard(self):
        grid_array = np.chararray((self.grid.height+1,self.grid.width+1),unicode=True)
        grid_array[:] = '.'
        for (i,j) in self.grid.floors():
            grid_array[j][i] = '-'
        for (i,j) in self.grid.corners():
            grid_array[j][i] = '+'
        for (i,j) in self.grid.walls():
            grid_array[j][i] = '|'
        for piece in self.resting_pieces:
            for (i,j) in piece.coords:
                grid_array[j][i] = '#'
        if self.current_piece:
            for (i,j) in self.current_piece.coords:
                grid_array[j][i] = '@'
        np.savetxt('aoc17gameboard.txt',np.flipud(grid_array),fmt='%s')

        
        

In [45]:
game_test = TetrisGame(7,example_list)

In [46]:
for i in range(2022):
    game_test.drop_piece()
game_test.draw_gameboard()

In [30]:
def height(gridsize,input_list,num_pieces):
    game = TetrisGame(gridsize,input_list)
    for i in range(num_pieces):
        game.drop_piece()
        i += 1
    return max([piece.top for piece in game.resting_pieces])
def all_cycle_states(gridsize,input_list,num_pieces):
    game = TetrisGame(gridsize,input_list)
    for i in range(num_pieces):
        game.drop_piece()
        i += 1
    return game.cycle_states


In [23]:
def search_for_cycle(cycle_states):
    tortoise = cycle_states[1]
    hare = cycle_states[2]
    try:
        i = 0
        while tortoise != hare:
            tortoise = cycle_states[1+i]
            hare = cycle_states[2*(1+i)]
            i += 1
        tort_index, hare_index = i,2*i

        mu = 0
        tortoise = cycle_states[0]
        while tortoise != hare:
            tortoise = cycle_states[mu]
            hare = cycle_states[hare_index + mu]
            mu += 1

        lamb = 1
        hare = cycle_states[1]
        while tortoise != hare:
            hare = cycle_states[mu + lamb]
            lamb += 1
        return(lamb, mu)
    except IndexError:
        print("Cycle Not Found")
        return None
    
def find_steps_for_cycle(lamb,mu,gridsize,input_list):
    mu_height =  height(gridsize,input_list,mu)
    lamb_height = height(gridsize,input_list,mu + lamb)
    return mu_height, lamb_height - mu_height

def find_height_via_cycle(gridsize,input_list,num_pieces):
    cycle_states = all_cycle_states(gridsize,input_list,10000)
    lamb, mu = search_for_cycle(cycle_states)
    mu_height, cycle_height_added = find_steps_for_cycle(lamb,mu,gridsize,input_list)
    div =int(np.floor((n - mu)/ lamb))
    remainder = (n - mu) % lamb
    print(div,remainder)
    r_height =  height(gridsize,input_list,mu + remainder) - mu_height
    print(mu_height,div,cycle_height_added,r_height)
    return mu_height + div*cycle_height_added + r_height

In [36]:
find_height_via_cycle(7,puzz_list,1000000000000)

579710144 1443
248 579710144 2702 2248


1566376811584

In [14]:


lamb, mu = search_for_cycle(newgame.cycle_states)
mu_height, cycle_height_added = find_steps_for_cycle(lamb,mu,7,example_list)
    

('horiz', 2, 38) ('horiz', 2, 38) 35
('horiz', 3, 6) ('horiz', 3, 6) 16
('horiz', 3, 6) ('horiz', 3, 6) 35
0


100%|█████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 4717.01it/s]


('horiz', 3, 6)
16


100%|█████████████████████████████████████████████████████████████████████████████████████████| 35/35 [00:00<00:00, 2197.78it/s]

('horiz', 3, 6)
51


In [24]:
def find_height(n,lamb, mu,mu_height,cycle_height_added,input_list):
    div =int(np.floor((n - mu)/ lamb))
    remainder = (n - mu) % lamb
    print(div,remainder)
    game = TetrisGame(7,input_list)
    for i in tqdm(range(remainder)):
        game.drop_piece()
        i += 1
    r_height =  max([piece.top for piece in game.resting_pieces])
    print(mu_height,div,cycle_height_added,r_height)
    return mu_height + div*cycle_height_added + r_height
n = 1000000000000
find_height(n,lamb, mu,mu_height,cycle_height_added,example_list)

28571428570 34


100%|█████████████████████████████████████████████████████████████████████████████████████████| 34/34 [00:00<00:00, 4344.97it/s]

26 28571428570 53 60


1514285714270

In [38]:
grid = Grid(7)
game_test = TetrisGame(grid,example_list)
game_test.draw_gameboard()

TypeError: unsupported operand type(s) for +: 'Grid' and 'int'

In [388]:
%%time
for i in tqdm(range(16)):
    game_test.drop_piece()
    i += 1
print(game_test.cycle_states[-1])
print(game_test.pieces_dropped)
max([piece.top for piece in game_test.resting_pieces])

100%|█████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 6306.63it/s]

('horiz', 3, 6)
16
CPU times: user 5.78 ms, sys: 2.03 ms, total: 7.81 ms
Wall time: 6.54 ms


26

In [321]:
%%time
for i in tqdm(range(34)):
    game_test.drop_piece()
    i += 1
print(game_test.cycle_states[-1])
print(game_test.pieces_dropped)
max([piece.top for piece in game_test.resting_pieces])

100%|█████████████████████████████████████████████████████████████████████████████████████████| 34/34 [00:00<00:00, 1189.89it/s]

('backL', 3, 23)
118
CPU times: user 30 ms, sys: 2.13 ms, total: 32.1 ms
Wall time: 31.5 ms


184

In [21]:
n = 1000000000000
div =int(np.floor((n - 16)/ 35))
remainder = (n - 16) % 35
print(div,remainder)

28571428570 34


In [323]:
26 + 28571428570*53 + 52

1514285714288

344

In [118]:
game_test.draw_gameboard()

In [119]:
max([piece.top for piece in game_test.resting_pieces])

116

In [120]:
def search_for_cycle(cycle_states):
    for i in range(int(len(cycle_states)/2-1)):
        tortoise = cycle_states[1+i]
        hare = cycle_states[2*(1+i)]
        if tortoise == hare:
            print(tortoise,hare,1+i)
            break

search_for_cycle(game_test.cycle_states)

('horiz', 2, 38) ('horiz', 2, 38) 35


53